## Проблеми наданої візуалізації

1. **Занадто багато різних типів даних в одному місці** - крім того, що для кожного регіогу є бар-чарти по 2 роках, то ще й для кожного року є різні класи різних кольорів.
2. **Відношення розмірів секторів барчарту** - неможливо співвіднести одні типи сфер для різних регіонів, оскільки кожен кольоровий сектор барчарту має різні основи. Тобто хоч якось ми можемо порівняти між регіонами ті сфери, які починаються з 0 (Сільське господарство). Всі інші - досить складно (чи можемо ми порівняти ВДВ Промисловості у Луганському решіоні і Полтавському? - дуже складано). І так само це заважає порівняти зміну у одній сфері одного регіону між роками, якщо це не Сільське господарство (наприклад різниця ВДВ Промисловості у Одеській області 2010 та 2017 року).
3. **Немає сортування (по областях)** - неможливо порівняти ВДВ щодо конкретної сфери у різних областях, на графіку дані виглядають дуже складно, не можливо помітити певний патерн між різними областями.
4. **Немає сортування (по сферах)** - неможливо порівняти різні сфери між собою, зрозуміти яка отримує загально більше/менше %.
5. **Позначки занадто далеко** - оскільки графік відносно широкий, позначки відсотків осі Y знаходиться доситься дадеко для регіонів, які не є першими п'ятьма, наприклад. Це не зручно для користувача - постійно повертатись поглядом щоб звірити значення.
6. **Легенда занадто далеко** - здається, складно співвідносити кольори і їх значення (хоч їх і 5), і це посилюється щільністю інформації. Мабуть, якби прямокутників з різними кольорами було 5, то це б працювало краще.
7. **Не видно загального патерну зміни між 2010 та 2017 роками** - неможливо порівняти 2 роки на жодному з рівнів (регіонів або сфер), данні відображення дуже щільно і неструктуровано для цього (хоч і є єдина послідовність - однаковий порядок відображення сфер у кожному регіоні).


## Нова візуалізація


In [101]:
import altair as al
import pandas as pd

In [102]:
df = pd.read_excel('./data.xlsx')

In [103]:
df.head()

,"Регіон, рік",сільське господарство,промисловість,будівництво,послуги,державне управління,рік
0,"Україна, 2010",8.3,25.9,3.3,57.3,5.2,2010
1,"Україна, 2017",12.1,25.2,2.5,53.7,6.5,2017
2,"Вінницька, 2010",21.6,19.3,4.1,47.4,7.6,2010
3,"Вінницька, 2017",28.1,21.4,2.0,39.8,8.7,2017
4,"Волинська, 2010",16.9,15.6,3.2,57.7,6.6,2010


In [104]:
new_region = []
for i in df['Регіон, рік'].iteritems():
  new_region.append(i[1].split(',')[0])

df['Регіон'] = new_region
df.drop(['Регіон, рік'], axis = 1, inplace = True)
df.columns = ['Agriculture', 'Industry', 'Construction', 'Services', 'Governance', 'Year', 'Region']

df.head()

,Agriculture,Industry,Construction,Services,Governance,Year,Region
0,8.3,25.9,3.3,57.3,5.2,2010,Україна
1,12.1,25.2,2.5,53.7,6.5,2017,Україна
2,21.6,19.3,4.1,47.4,7.6,2010,Вінницька
3,28.1,21.4,2.0,39.8,8.7,2017,Вінницька
4,16.9,15.6,3.2,57.7,6.6,2010,Волинська


In [105]:
all_types = ['Agriculture', 'Industry', 'Construction', 'Services', 'Governance']
ukranians = [
  'Сільске, лісове та рибне господарство',
  'Промисловість',
  'Будівництво',
  'Послуги',
  'Державне управління й оборона']
finished = []

max_percentage_ever = max([df[way].max() for way in all_types])

select_region = alt.selection_single(on = 'mouseover', empty = 'all',  fields=['Region'])
for way in all_types:
  lines = alt.Chart(df).mark_line().encode(
    y = alt.Y(f'max({way}):Q', scale = alt.Scale(zero = False,
                                domain = [0, max_percentage_ever]), title='%'),
    y2 = alt.Y2(f'min({way}):Q'),
    x = alt.X('Region:N', sort=alt.Sort(field=way, op='max', order='descending')),
    strokeWidth = alt.value(5),
    color = alt.condition(select_region, alt.ColorValue('#34495E'), alt.ColorValue('#D5D8DC'))
    ).add_selection(
    select_region
).encode(tooltip = [
        alt.Tooltip('Region:N'),
        alt.Tooltip(f'max({way}):Q'),
        alt.Tooltip(f'min({way}):Q'),
    ])

  points = alt.Chart(df).mark_square(size = 60).encode(
    x = alt.Y('Region:N', sort=alt.Sort(field=way, op='max', order='descending')),
    y = alt.X(f'{way}:Q'),
    color  = alt.Color('Year:N', scale=alt.Scale(range=['#48C9B0', '#F1C40F']))
).transform_filter(select_region)

  finished_chart = alt.layer(lines, points).properties(width = 800, height = 600,
   title = alt.TitleParams(
    text = ukranians[all_types.index(way)])
  )

  finished.append(finished_chart)

In [106]:
column_width = 250
column_height = 500

alt.hconcat(finished[3].properties(width = column_width, height = column_height),
            finished[1].properties(width = column_width, height = column_height),
            finished[0].properties(width = column_width, height = column_height),
            finished[4].properties(width = column_width, height = column_height),
            finished[2].properties(width = column_width, height = column_height)
           ).add_selection(select_region).properties(background = '#F9F9F9', padding = 45, title = alt.TitleParams(
    text = 'Зміни структури ВДВ у різних регіонах в розрізі різних сфер за 2017 рік порівняно з 2010 роком',
    fontSize=24, dx=150, dy=-40)
  ).configure_title(
    anchor = 'start',
    frame = 'group',
    fontSize = 14,
    subtitleFontSize = 16
)

alt.HConcatChart(...)